<a name='top'></a>    
<div style="width:900px;background:#fdf0db;border:1px solid black;text-align:left;padding:8px;">
    <p>
        <span style="font-size:14pt">
            <b>Scraping from www.mycareersfuture.sg (Detail Page)</b>
        </span>
    </p>
</div>

<div style="width:900px;background:#f2f2f2;border:1px solid black;text-align:left;padding:8px;">

This notebook file will extract detail information from Job Details extracted from the 1st Notebook file,(01-webscrape-pages.ipynb).

The Details such as Job Description and Job Requirements will be extracted to further help extract more details for analysis.  


In order to run this project please execute the following:
1. Run 01-webscrape-pages.ipynb
2. Run 02-webscrape-details.ipynb
3. Move all output files created by these two notebooks to data folder.
4. Run 03-data-val-and-cleaning.ipynb
   - This notebook will create a final csv file in data folder.
5. Run 04-data-modelling.ipynb
   - This notebook will read the final csv file to perform analysis.
   
Please note that All output files produced by the first two notebooks (01-webscrape-pages.ipynb & 02-webscrape-details.ipynb) will be written to the current directory.  

In [10]:
import os
from selenium import webdriver
# Import sleep:
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re

In [11]:
## Generate a list of page files to be processed (remove the .csv extension)
myfiles=['outfile_0{}'.format(x) for x in range(6)]  

for x in range(5):
    myfiles.append('outfile_a0{}'.format(x))
    
print(myfiles[:6])
print(myfiles[6:20])

['outfile_00', 'outfile_01', 'outfile_02', 'outfile_03', 'outfile_04', 'outfile_05']
['outfile_a00', 'outfile_a01', 'outfile_a02', 'outfile_a03', 'outfile_a04']


In [27]:
# Run this to extract process one file.  Used mostly for testing purposes.
#html=webscraper_detail(myfiles[:2])

In [ ]:
## This function will extract all page files and call process_links()
def webscraper_detail(files):
    #print(files,'\n')
    for file in files:
        file_in  =file +'.csv'
        file_out =file +'_detail.csv'
        
        df = pd.read_csv(file_in, encoding='latin1')
        df2=df[['links','location']]

        #html= process_links(df2[:2], file_in, file_out)  ## raj testing
        html=process_links(df2, file_in, file_out)                
    return html

In [ ]:
## This function will extract details and output to new csv file.
def process_links(df, file_in, file_out='test_output.csv'):
    #chromedriver = "chromedriver.exe"
    chromedriver = r"C:\Users\roni_\Documents\1Roni\GA\Install\chromedriver\chromedriver.exe"
    os.environ["webdriver.chrome.driver"] = chromedriver

    driver = webdriver.Chrome(executable_path=chromedriver)

    ref=[]
    job_titles=[]
    companys=[]
    employment_types=[]
    senioritys=[]
    industrys=[]
    salary=[]
    salary_terms=[]
    job_descriptions=[]
    job_requirements=[]
    weblinks=[]
    location=[]
    gov_support=[]

    rec_count=0
    range_count=0

    print('Processing',file_in,':')
    
    for link, loc in df.itertuples(index=False):
        ref.append(link)
        driver.get(link)
        sleep(8)
        # Grab the page source.div class="bg-white pa4 mb3"
        html = driver.page_source
        # Beautiful Soup it!
        html = BeautifulSoup(html, 'lxml')

        # Location
        location.append(loc)
        # gov_support
        gov_supp='NA'
        for entry in html.find_all('section', {'class':'jobSchemes w-100 mb3 dib tl pa4 bg-washed-yellow bt bw2 b--light-yellow'}):
            gov_supp=entry.find('p',{'class':'f5 fw7 pa0 mh0 mt0 mb4 brand-sec'}).text

        if gov_supp == 'NA':
            ## NO Govt Support Section Found
            gov_support.append(np.nan)
        else:
            gov_support.append(gov_supp)

        # search for the rest of the fields
        for entry in html.find_all('div', {'class':'bg-white pa4 mb3'}):

            #Grab the Jobtitle.
            job_title = entry.find('h1', {'class': 'f3 fw6 mv0 pv0 mb1 dark-pink w-100 dib'}).text
            job_titles.append(job_title)
            #print(job_title)

            # Grab the Company.
            try:
                company = entry.find('p', {'class': 'f6 fw6 mv0 black-80 mr2 di ttu'}).text
                companys.append(company)
            except:
                companys.append(np.nan)

            #seniorty
            try:
                seniority=entry.find('p',{'class': 'black-60 f6 fw4 mv1 dib pr3 mr1 icon-bw-seniority'}).text
                senioritys.append(seniority)
            except:
                senioritys.append(np.nan)

            #employment_type
            try:
                employment_type=entry.find('p',{'class':'black-60 f6 fw4 mv1 dib pr3 mr1 icon-bw-employment-type'}).text
                employment_types.append(employment_type)
            except:
                employment_types.append(np.nan)

            #industry:
            try:
                industry=entry.find('p',{'class':'black-60 f6 fw4 mv1 dib pr3 mr1 icon-bw-category'}).text
                industrys.append(industry)
            except:
                industrys.append(np.nan)

            #salary:
            for row in entry.findAll("span", class_ = re.compile("salary_range")):
                salary.append(row.text)

            #monthly/yearly
            try:
                salary_term=entry.find('span', {'class':'salary_type dib f5 fw4 black-60 pr1 i pb'}).text
                salary_terms.append(salary_term)
            except:
                salary_terms.append(np.nan)

            #job description
            try:
                job_description=entry.find('div', {'id': 'description-content'}).text
                job_descriptions.append(job_description)
            except:
                job_descriptions.append(np.nan)

            #requirements
            try:
                job_requirement=entry.find('div', {'id': 'requirements'}).text
                job_requirements.append(job_requirement)
            except:
                job_requirements.append(np.nan)

            rec_count = rec_count + 1
            range_count = range_count + 1

            if range_count == 10:
                print ('   processing', rec_count, 'records')
                range_count = 0

            #print('   ', rec_count,':',gov_supp,job_title)

    # First, create an empty DataFrame.
    import pandas as pd

    column_headers=["job_titles", "companys", "employment_type", "seniority",
             "industrys", "salary", "salary_terms", "job_descriptions",
             "job_requirements", "links", "location", "gov_support"]

    jobs = pd.DataFrame(columns= column_headers)

    jobs["job_titles"]=job_titles
    jobs["companys"]=companys
    jobs["employment_type"]=employment_types
    jobs["seniority"]= senioritys
    jobs["gov_support"]=gov_support
    jobs["industrys"]=industrys
    jobs["salary"]=salary
    jobs["salary_terms"]=salary_terms
    jobs["job_descriptions"]=job_descriptions
    jobs["job_requirements"]=job_requirements
    jobs["links"]=ref
    jobs["location"]=location

    jobs.to_csv(file_out)
    driver.close()
    print('==========================')
    print('Records Written({}):'.format(file_out),rec_count)
    print('==========================')
    print()
    
    return html

In [ ]:
## Call function to webscrape from detail files.

In [35]:
html=webscraper_detail(myfiles[3:6])

['outfile_03', 'outfile_04', 'outfile_05'] 

Processing outfile_03.csv :
   processing 10 records
   processing 20 records
   processing 30 records
   processing 40 records
   processing 50 records
   processing 60 records
   processing 70 records
   processing 80 records
   processing 90 records
   processing 100 records
Records Written(outfile_03_detail.csv): 104

Processing outfile_04.csv :
   processing 10 records
   processing 20 records
   processing 30 records
   processing 40 records
   processing 50 records
   processing 60 records
   processing 70 records
   processing 80 records
   processing 90 records
   processing 100 records
   processing 110 records
   processing 120 records
   processing 130 records
   processing 140 records
Records Written(outfile_04_detail.csv): 147

Processing outfile_05.csv :
   processing 10 records
   processing 20 records
   processing 30 records
   processing 40 records
   processing 50 records
   processing 60 records
   processing 70 records


In [48]:
html=webscraper_detail([myfiles[7]])

['outfile_a01'] 

Processing outfile_a01.csv :
   processing 10 records
   processing 20 records
   processing 30 records
   processing 40 records
   processing 50 records
   processing 60 records
   processing 70 records
   processing 80 records
   processing 90 records
   processing 100 records
   processing 110 records
   processing 120 records
   processing 130 records
   processing 140 records
   processing 150 records
   processing 160 records
   processing 170 records
   processing 180 records
   processing 190 records
   processing 200 records
   processing 210 records
   processing 220 records
   processing 230 records
   processing 240 records
   processing 250 records
   processing 260 records
Records Written(outfile_a01_detail.csv): 266



In [49]:
html=webscraper_detail([myfiles[8]])

['outfile_a02'] 

Processing outfile_a02.csv :
   processing 10 records
   processing 20 records
   processing 30 records
   processing 40 records
   processing 50 records
   processing 60 records
   processing 70 records
   processing 80 records
   processing 90 records
   processing 100 records
   processing 110 records
   processing 120 records
   processing 130 records
   processing 140 records
   processing 150 records
   processing 160 records
   processing 170 records
   processing 180 records
   processing 190 records
   processing 200 records
   processing 210 records
   processing 220 records
   processing 230 records
   processing 240 records
   processing 250 records
   processing 260 records
Records Written(outfile_a02_detail.csv): 260



In [52]:
html=webscraper_detail([myfiles[9]])

['outfile_a03'] 

Processing outfile_a03.csv :
   processing 10 records
   processing 20 records
   processing 30 records
   processing 40 records
   processing 50 records
   processing 60 records
   processing 70 records
   processing 80 records
   processing 90 records
   processing 100 records
Records Written(outfile_a03_detail.csv): 100



In [53]:
html=webscraper_detail([myfiles[10]])

['outfile_a04'] 

Processing outfile_a04.csv :
   processing 10 records
Records Written(outfile_a04_detail.csv): 19

